In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from extract_feature import *

In [63]:
dataset = {}
dataset['paper'] = pd.read_csv('dataRev2/Paper.csv').fillna("")
dataset['author'] = pd.read_csv('dataRev2/Author.csv').fillna("")
dataset['conference'] = pd.read_csv('dataRev2/Conference.csv').fillna("")
dataset['journal'] = pd.read_csv('dataRev2/Journal.csv').fillna("")
dataset['paper_author'] = pd.read_csv('dataRev2/PaperAuthor.csv').fillna("")

In [102]:
def generate_feature_list(author_paper_pairs, ap_to_feature_list):
    result_list = []

    temp_dict = {} # { (author, paper) => [f1, f2 ...] }
    for ap_pair in author_paper_pairs:
        temp_dict[ap_pair] = []

    for i in range(len(ap_to_feature_list)):
        feature_dict = ap_to_feature_list[i]
        for ap_pair in author_paper_pairs:
            feature = feature_dict[ap_pair]
            temp_dict[ap_pair].append(feature)

    for key in temp_dict.keys():
        result_list.append(key + tuple( temp_dict[key] ))

    return result_list

def get_coauthor_aff(pa_data,pid):
    related_authors = pa_data[pa_data['PaperId'] == pid]['Affiliation']
    return related_authors

def kamil_feature_11(dataset, author_paper_pairs):
    feature_dict = {}
    pa_data = dataset["paper_author"]
    print(author_paper_pairs[:2])
    for ap in author_paper_pairs:
        pa = ap[::-1]
        print("PA pair",pa)
        paper_aff = pa_data[(pa_data["AuthorId"] == pa[1]) & (pa_data["PaperId"] == pa[0])]["Affiliation"].unique()
        print("Paper aff--",paper_aff,"--")
        
        ta_aff = " ".join(pa_data[pa_data["AuthorId"] == pa[1]]["Affiliation"].unique())
        print("Target aff--",ta_aff,"--")
        coa_aff = get_coauthor_aff(pa_data, pa[0])
        value = 0
        for i,aff in enumerate(coa_aff):
            if(aff != ""):
                print(i,"Co auth aff::",aff,"::")
                value += lcs(aff, ta_aff)
        feature_dict[pa] = value
        print(pa," has ",value)
    return feature_dict

def get_features(dataset, targetset):
    author_paper_pairs = parse_targetset(targetset)

    # Keep the format of f# (dictionary): { (a1, p1): feature_value1, (a2, p2): feature_value2 ... }
    # Add your features here and add them to feature_list!
    kamil_f1 = kamil_feature_2(dataset, author_paper_pairs)
    kamil_list = [kamil_f1]
    
    thao_list,harry_list = [],[]
    feature_list = harry_list + kamil_list + thao_list

    result_list = generate_feature_list(author_paper_pairs, feature_list)
    return result_list

In [103]:
sample_l = [(1539933, 1359549), (1455231, 467172), (1215636, 1791266), (205278, 1737961), (433821, 1901940)]
kamil_feature_11(dataset,sample_l)

[(1539933, 1359549), (1455231, 467172)]
PA pair (1359549, 1539933)
Paper aff-- ['Hokkaido University' ''] --
Target aff--  Hokkaido University Institute for Solid State Physics, University of Tokyo, Kashiwanoha 5-1-5, Kashiwa-shi, Chiba 277-8581, Japan Department of Physics, Faculty of Science, University of Tokyo, 7-3-1 Hongo, Bankyo-ku, Tokyo 113, Japan Institute for Solid State Physics, University of Tokyo, 5-1-5 Kashiwanoha, Kashiwa, 277-8581 Japan Institute for Solid State Physics University of Tokyo 5-1-5 Kashiwanoha 277-8581 Kashiwa Japan Dep. of Phys. Univ. of Tokyo, Tokyo 113, Japan Department of Physics, Faculty of Science, the University of Tokyo, Hongo, Bunkyo-ku, Tokyo 113-8654, Japan Institute for Solid State Physics University of Tokyo 5-1-5 Kashiwanona Kashiwa 277-8581 Japan --
4 Co auth aff:: Hokkaido University ::
(1359549, 1539933)  has  19.0
PA pair (467172, 1455231)
Paper aff-- ['Beijing Jiaotong University'
 'Institute of Information Science|Beijing Jiaotong Unive

{(467172, 1455231): 323.0,
 (1359549, 1539933): 19.0,
 (1737961, 205278): 0,
 (1791266, 1215636): 46.0,
 (1901940, 433821): 137.0}

In [21]:
features_conf = get_features(dataset, train_confirmed)
features_conf[:5]
#train_confirmed.head()

[(1539933, 1359549), (1455231, 467172), (1215636, 1791266), (205278, 1737961), (433821, 1901940)]


NameError: name 'pa_only' is not defined

In [98]:
def lcs(X , Y):
    # find the length of the strings
    m = len(X)
    n = len(Y)
 
    # declaring the array for storing the dp values
    L = np.zeros((m+1,n+1))
 
    """Following steps build L[m+1][n+1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m+1):
        for j in range(n+1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X[i-1] == Y[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j] , L[i][j-1])
 
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1]
    return L[m][n]
lcs("ababababababababb","bbbbbb")

6.0

In [34]:
def find_relation(author_id,paper_id):
    relation_list = pd.DataFrame()
    paper_author = dataset['paper_author']
    pa_only = dataset['paper_author'][['PaperId','AuthorId']]
    #find papers related to the author
    related_papers = pa_only[pa_only.AuthorId == author_id]['PaperId'].unique()
    print("# related papers:",related_papers.shape)
    
    #find authors related to the paper
    related_authors = paper_author[paper_author['PaperId'] == paper_id]['AuthorId'].unique()
    print("# related authors:",related_authors.shape)
    
    #remove target paper and target author from lists
    related_papers = np.delete(related_papers,np.argwhere(related_papers == paper_id))
    related_authors = np.delete(related_authors, np.argwhere(related_authors == author_id))
    
    print("# related papers:",related_papers.shape)
    print("# related authors:",related_authors.shape)
    cnt = 0
    for author in related_authors:
        for paper in related_papers:
            single_relation = pa_only[(pa_only['PaperId'] == paper) & (pa_only['AuthorId'] == author)]
            cnt += single_relation.size
            
    print(author_id,paper_id,"->",cnt)
    
    return cnt


In [35]:
%time find_relation(826,25733)

# related papers: (92,)
# related authors: (6,)
# related papers: (91,)
# related authors: (5,)
826 25733 -> 96
Wall time: 29.6 s


96